# 01 - Reconstruction

Reconstruct speed, acceleration, direction, and other derived signals from position data.


Let’s now build a **step-by-step guided version** —
think of it as your **“Merge & Reconstruction Playbook”** 🧠⚙️ —
with each step having:

* 🎯 *Purpose (why we do it)*
* 🧩 *Action (what exactly to code/do)*
* 🧠 *Validation (how to confirm it worked)*

---

# 🧭 MERGE & RECONSTRUCTION PLAYBOOK

**Goal:** Safely merge `input`, `output`, and `supplementary` tracking data
into one continuous, consistent dataset ready for metric computation.

---

## 🔹 Step 1 — Load and Inspect

**🎯 Purpose:**
Get familiar with what you’re merging — shape, columns, basic counts.

**🧩 Action:**

```python
import pandas as pd
from pathlib import Path

input_df = pd.read_parquet('data/raw/week1_input.parquet')
output_df = pd.read_parquet('data/raw/week1_output.parquet')
supp_df = pd.read_csv('data/raw/supplementary_data.csv')

print(input_df.shape, output_df.shape, supp_df.shape)
print(input_df.columns)
```

**🧠 Validation:**
✅ Files load without error.
✅ Columns match expectations (`game_id`, `play_id`, `nfl_id`, etc.).

---

## 🔹 Step 2 — Validate Keys & Uniqueness

**🎯 Purpose:**
Prevent silent ID mixups that could ruin reconstruction.

**🧩 Action:**

```python
print("Input duplicates:", input_df.duplicated(['game_id','play_id','nfl_id','frame_id']).sum())
print("Output duplicates:", output_df.duplicated(['game_id','play_id','nfl_id','frame_id']).sum())
print("Supplementary duplicates:", supp_df.duplicated(['game_id','play_id']).sum())
```

**🧠 Validation:**
✅ All zero (or very low if legitimate).
✅ Any duplicates should be inspected — they mean repeated records for same frame/player.

---

## 🔹 Step 3 — Attribute Extraction

**🎯 Purpose:**
We don’t want to lose player metadata (height, role, etc.) during merge.

**🧩 Action:**

```python
player_attrs = input_df[['game_id','play_id','nfl_id','player_name','player_height',
                         'player_weight','player_birth_date','player_position','player_side',
                         'player_role','num_frames_output','ball_land_x','ball_land_y']].drop_duplicates()
```

**🧠 Validation:**
✅ Each `(game_id, play_id, nfl_id)` unique.
✅ No missing critical info (like player_name).

---

## 🔹 Step 4 — Frame Continuity & Normalization

**🎯 Purpose:**
Output data restarts frame numbering at 1 → we must realign them so
input + output form one continuous timeline.

**🧩 Action:**

```python
# Find max input frame for each player per play
max_input_frame = input_df.groupby(['game_id','play_id','nfl_id'])['frame_id'].max().reset_index()
max_input_frame.rename(columns={'frame_id':'last_input_frame'}, inplace=True)

# Merge into output and shift frame_ids
output_df = output_df.merge(max_input_frame, on=['game_id','play_id','nfl_id'], how='left')
output_df['frame_id_shifted'] = output_df['frame_id'] + output_df['last_input_frame']
```

**🧠 Validation:**
✅ Each player’s timeline continues smoothly (no overlaps).
✅ `frame_id_shifted` increases after input frames.

---

## 🔹 Step 5 — Merge Attributes Into Frames

**🎯 Purpose:**
Ensure both input and output frames carry static player information.

**🧩 Action:**

```python
input_df = input_df.merge(player_attrs, on=['game_id','play_id','nfl_id'], how='left')
output_df = output_df.merge(player_attrs, on=['game_id','play_id','nfl_id'], how='left')
```

**🧠 Validation:**
✅ After merge, both have equal player-level fields.
✅ No unexpected NaNs introduced.

---

## 🔹 Step 6 — Concatenate Input + Output

**🎯 Purpose:**
Build a single, chronological sequence from pre-throw → post-throw.

**🧩 Action:**

```python
output_df = output_df.drop(columns=['frame_id']).rename(columns={'frame_id_shifted':'frame_id'})
combined_df = pd.concat([input_df, output_df], ignore_index=True)
combined_df.sort_values(['game_id','play_id','nfl_id','frame_id'], inplace=True)
```

**🧠 Validation:**
✅ Each player’s frames are sequential and increasing.
✅ Plot one player’s x–y over time to visually confirm smoothness.

---

## 🔹 Step 7 — Merge Play Outcomes

**🎯 Purpose:**
Attach the play results (e.g., complete, incomplete) from supplementary data.

**🧩 Action:**

```python
reconstructed_df = combined_df.merge(supp_df, on=['game_id','play_id'], how='left', validate='m:1')
```

**🧠 Validation:**
✅ No mismatched or missing plays.
✅ Number of rows unchanged.

---

## 🔹 Step 8 — Quality Evaluation

**🎯 Purpose:**
Confirm data integrity after full reconstruction.

**🧩 Action:**

```python
print("Missing data ratio:\n", reconstructed_df.isna().mean())
print("Unique player count check:",
      len(reconstructed_df['nfl_id'].unique()),
      "vs", len(input_df['nfl_id'].unique()))

# Check continuity per player
gap_check = reconstructed_df.groupby(['game_id','play_id','nfl_id'])['frame_id'].diff().dropna()
print("Frame gaps:", (gap_check > 1).sum())
```

**🧠 Validation:**
✅ Missing < 1–2% (mostly tolerable metadata).
✅ Frame gaps = 0 (continuous motion).

---

## 🔹 Step 9 — Visual Verification (Optional)

**🎯 Purpose:**
Human-eye validation to confirm continuity of player trajectories.

**🧩 Action:**

```python
import matplotlib.pyplot as plt

sample = reconstructed_df.query("game_id==2023091001 and play_id==4216")
for pid in sample['nfl_id'].unique():
    p = sample[sample['nfl_id']==pid]
    plt.plot(p['x'], p['y'], label=pid)
plt.title("Reconstructed Player Motion")
plt.legend()
plt.show()
```

**🧠 Validation:**
✅ Smooth player paths → no teleportation (input→output transition continuous).

---

## 🔹 Step 10 — Save & Document

**🎯 Purpose:**
Persist clean, validated reconstruction for feature engineering.

**🧩 Action:**

```python
output_path = Path("data/processed/reconstructed.parquet")
reconstructed_df.to_parquet(output_path, index=False, compression='snappy')
```

**🧠 Validation:**
✅ File saved, correct row count.
✅ Add log entry:

```python
print(f"✅ Reconstruction complete — {len(reconstructed_df)} rows saved at {output_path}")
```

---

## 🧾 End Deliverable

✅ `data/processed/reconstructed.parquet`
✅ Merge validation summary (NaNs, frame continuity, sample plot)
